##### this notebooks has the sample code for the qna generation functions and calculations checks with optimized code.

In [5]:
%load_ext autoreload
%autoreload 2

The autoreload extension is already loaded. To reload it, use:
  %reload_ext autoreload


In [6]:
import os
curpath = os.getcwd()
os.chdir(curpath.split("core")[0])

In [33]:
import openai
import json

from prompts import SIMPLE_NUMERICAL_MCQ_PROMPT, SIMPLE_TEXTUAL_TEXT_MCQ_PROMPT, SIMPLE_CONCEPTUAL_TEXT_MCQ_PROMPT, CALCULATION_CHECK_PROMPT
from helper import function_definitions, get_results_from_wolfram_cloud
from utils import add_dicts

from dotenv import load_dotenv
load_dotenv()

True

### Functions

In [34]:
functions = [function_definitions["get_results_from_wolfram_cloud"]] 

functions

[{'name': 'get_results_from_wolfram_cloud',
  'description': 'Use this function for problems solvable with Wolfram Language code.',
  'parameters': {'type': 'object',
   'properties': {'queries': {'type': 'string',
     'description': 'the input wolfram language query for mathematical calculations in string. ONLY proper wolfram language queries are supported. Format for Query: "a = Solve[aCoeff1*aVar == Var1 - Offset1 && aCoeff2*aVar == Var1 + Offset2, {aVar, Var1}][[1, 1, 2]]".'}}},
  'required': ['queries']}]

### Conceptual Text MCQ Generation function

In [8]:
### theory will be the context text for conceptual questions. context will be used for textual/assessment type questions.

def generate_conceptual_text_mcqs(theory=None, n=1, examples = None, model_id = "gpt-4-0613"):

    conversation = [{"role": "system", "content": SIMPLE_CONCEPTUAL_TEXT_MCQ_PROMPT}]

    if theory is None:
        raise ValueError("No theory provided")

    user_prompt = f"""
    TOPIC_THEORY:
    //theory//

    {theory}

    //theory//
   
    N: {n}

    EXAMPLE:
    {examples if examples else "Not Provided"}

    QUESTIONS: 
    """

    user_message = {'role': 'user', "content": user_prompt}
    conversation.append(user_message)
    
    total_usage = {'prompt_tokens': 0, 'completion_tokens': 0, 'total_tokens': 0}
    response = openai.ChatCompletion.create(
                                    model= model_id,
                                    messages = conversation,
                                    temperature = 1
                                    )
    
    total_usage = add_dicts(total_usage, dict(response.usage))
    output = response.choices[0].message.content
    
    return {"output": output, "total_usage": total_usage}

In [ ]:
theory = """
Thermodynamics

Definitions:
1. System: A part of the universe under study.
2. Surroundings: Everything other than the system.
3. Thermodynamic State: The description of a system in terms of properties like pressure, volume, temperature, etc.
4. State Functions: Properties that depend only on the state of the system, not the path. Examples: U (Internal energy), H (Enthalpy), S (Entropy), and G (Gibbs free energy).
5. Path Functions: Depend on the path taken during a change, like heat (q) and work (w).
6. First Law of Thermodynamics: ΔU = q + w, where ΔU is the change in internal energy, q is heat, and w is work.
7. Isothermal Process: Process in which temperature remains constant.
8. Adiabatic Process: No heat exchange with surroundings. 

"""

examples = {
        "question": "Which of the following statements is true regarding thermodynamics?",
        "choices": {"a": "The surroundings refer to the part of the universe under study.", "b": "State Functions depend on the path taken during a change.", "c": "In an adiabatic process, there is no heat exchange with the surroundings.", "d": "The First Law of Thermodynamics is represented by the equation ΔU = q - w."},
        "answer": {"c": "In an adiabatic process, there is no heat exchange with the surroundings"}
    }

output = generate_conceptual_text_mcqs(
                        theory = theory,
                        n = 3
                        # examples = examples,
                        )

print(output['output'])

### Textual Text MCQ Generation function

In [11]:

# context will be the context text for textual questions. context will be used for textual/assessment type questions.
def generate_textual_text_mcqs(context=None, n=1, examples = None, model_id = "gpt-4-0613"):

    if theory is None:
        raise ValueError("context must be provided")
    

    conversation = [{"role": "system", "content": SIMPLE_TEXTUAL_TEXT_MCQ_PROMPT}]

    user_prompt = f"""
    CONTEXT_TEXT:
    //context//

    {context}

    //context//

    N: {n}

    EXAMPLES:
    {str(examples) if examples else "Not Provided"}

    QUESTIONS: 
    """

    user_message = {'role': 'user', "content": user_prompt}
    conversation.append(user_message)

    total_usage = {'prompt_tokens': 0, 'completion_tokens': 0, 'total_tokens': 0}
    response = openai.ChatCompletion.create(
                                    model= model_id,
                                    messages = conversation,
                                    temperature = 1
                                    )
    
    total_usage = add_dicts(total_usage, dict(response.usage))

    output = response.choices[0].message.content

    return {"output": output, "total_usage": total_usage}


In [12]:
context = """Many United States companies have, unfortunately, made the search for legal protection from import competition into a major line of work. Since 1980 the United States International Trade Commission (ITC) has received about 280 complaints alleging damage from imports that benefit foreign governments’ subsidies. Another 340 charge that foreign companies “dumped” their products in the United States at “less than fair value.” Even when no unfair practices are alleged, the simple claim that an industry has been injured by imports is sufficient grounds to seek relief.

Contrary to the general impression, this quest for import relief has hurt more companies than it has helped. As corporations begin to function globally, they develop an intricate web of marketing, production, and research relationships, The complexity of these relationships makes it unlikely that a system of import relief laws will meet the strategic needs of all the units under the same parent company.

Internationalization increases the danger that foreign companies will use import relief laws against the very companies the laws were designed to protect. Suppose a United States-owned company establishes an overseas plant to manufacture a product while its competitor makes the same product in the United States. If the competitor can prove injury from the imports—and that the United States company received a subsidy from a foreign government to build its plant abroad—the United States company’s products will be uncompetitive in the United States, since they would be subject to duties.

Perhaps the most brazen case occurred when the ITC investigated allegations that Canadian companies were injuring the United States salt industry by dumping rock salt, used to de-ice roads. The bizarre aspect of the complaint was that a foreign conglomerate with United States operations was crying for help against a United States company with foreign operations. The “United States” company claiming the injury was a subsidiary of a Dutch conglomerate. In contrast, the “Canadian” companies included a subsidiary of a Chicago firm that was the second-largest domestic producer of rock salt."""

examples = {
        "question": "Which of the following options is most similar in meaning to the word: “Intricate” mentioned in the article?",
        "choices": {"a": "Twisted", "b": "Straightforward", "c": "Simple", "d": "Advance"},
        "answer": {"a": "Twisted"}
    }

result = generate_textual_text_mcqs(context, n = 2, examples = examples)

print(result["output"])
print(result["total_usage"])


    {
        "mcqs": [
            {
            "question": "What is the main problem raised by the author regarding the use of import relief laws?",
            "choices": {"a": "They benefit foreign governments.", "b": "They may be used against U.S. companies by foreign competitors.", "c": "They discourage companies from establishing overseas plants.", "d": "They prevent domestic companies from dumping their products at less than fair value."},
            "answer": {"b": "They may be used against U.S. companies by foreign competitors."}
            },
            {
            "question": "Who was claiming injury in the case of dumping rock salt by Canadian companies?",
            "choices": {"a": "A U.S. company with overseas operations", "b": "A foreign conglomerate with U.S. operations", "c": "A Canadian company", "d": "A Chicago-based firm"},
            "answer": {"b": "A foreign conglomerate with U.S. operations"}
            }
        ]
    }


In [13]:
print(result["total_usage"])

{'total_tokens': 1142, 'prompt_tokens': 931, 'completion_tokens': 211}


### Numerical MCQ Generation function

In [19]:

def generate_numerical_mcqs(theory=None, n=1, examples = None, model_id = "gpt-4-0613"):

    if theory is None:
        raise ValueError("Theory must be provided")

    conversation = [{"role": "system", "content": SIMPLE_NUMERICAL_MCQ_PROMPT}]

    user_prompt = f"""
    TOPIC_THEORY:
    //theory//

    {theory}

    //theory//

    N: {n}

    EXAMPLES:
    {str(examples) if examples else "Not Provided"}

    QUESTIONS: 
    """

    user_message = {'role': 'user', "content": user_prompt}
    conversation.append(user_message)

    total_usage = {'prompt_tokens': 0, 'completion_tokens': 0, 'total_tokens': 0}
    response = openai.ChatCompletion.create(
                                    model= model_id,
                                    messages = conversation,
                                    temperature = 1
                                    )
    
    total_usage = add_dicts(total_usage, dict(response.usage))

    output = response.choices[0].message.content

    return {"output": output, "total_usage": total_usage}


In [20]:
theory = """
Work Done in Various Processes:
a. Isothermal Process (constant temperature):
W = P(V2 - V1) * ln(V2 / V1)
b. Adiabatic Process (no heat exchange):
W = (P2V2 - P1V1) / (1 - γ), where γ is the adiabatic index
c. Isobaric Process (constant pressure):
W = P(V2 - V1)"""

examples = {
        "question": """Consider three processes: isothermal, adiabatic, and isobaric. For a certain gas, the following data is given:

Initial volume, V1 = 2 m^3
Final volume, V2 = 4 m^3
Initial pressure, P1 = 2 atm
Final pressure, P2 = 3 atm
Adiabatic index, γ = 1.4
Calculate the work done during the adiabatic process.""",
        "choices": {"a": "-1.5 atm.m^3", "b": "-2.0 atm.m^3", "c": "-2.5 atm.m^3", "d": "-20.0 atm.m^3"},
        "answer": {"d": "-20.0 atm.m^3"}
    }

result = generate_numerical_mcqs(theory = theory, n = 2, examples = examples)

In [24]:
print(result["output"])
print(result['total_usage'])

json_ques = json.loads(result['output'])

{
    "mcqs": [
        {
        "question": "Calculate the work done during an isothermal process if the initial volume (V1) is 1 m^3, the final volume (V2) is 3 m^3 and the pressure is 1 atm.",
        "choices": {"a": "0.367 atm.m^3", "b": "0.693 atm.m^3", "c": "1.098 atm.m^3", "d": "1.386 atm.m^3"},
        "answer": {"b": "0.693 atm.m^3"}
        },
        {
        "question": "Calculate the work done during an isobaric process if the initial volume (V1) is 1 m^3, the final volume (V2) is 5 m^3 and the pressure is 1 atm.",
        "choices": {"a": "1 atm.m^3", "b": "2 atm.m^3", "c": "3 atm.m^3", "d": "4 atm.m^3"},
        "answer": {"d": "4 atm.m^3"}
        }
    ]
}
{'total_tokens': 905, 'prompt_tokens': 660, 'completion_tokens': 245}


#### calculation check for numerical mcqs

In [35]:
# calculation check and improvements

def calculation_checking(input, model_id = "gpt-4-0613"):

    conversation = [{"role": "system", "content": CALCULATION_CHECK_PROMPT}]
    input_prompt = f"""

    INPUT: 

    {input}

    OUTPUT: 

    """ 

    input_message = {'role': 'user', 'content': input_prompt}
    conversation.append(input_message)

    total_usage = {'prompt_tokens': 0, 'completion_tokens': 0, 'total_tokens': 0}
    response = openai.ChatCompletion.create(
                                    model= model_id,
                                    messages = conversation,
                                    functions = functions,
                                    function_call = {'name': "get_results_from_wolfram_cloud"},
                                    temperature = 0
                                )

    total_usage = add_dicts(total_usage, dict(response.usage))
    function_call = response.choices[0]['message']['function_call']
    conversation.append(response.choices[0]['message'])

    queries = json.loads(function_call["arguments"])["queries"]
    try:
        print("good queries", queries)
        result = get_results_from_wolfram_cloud(queries)
    except:
        raise Exception("error from wolfram")

    conversation.append({'role': 'function', "name": function_call["name"], "content": str(result)})

    response = openai.ChatCompletion.create(
        model = model_id,
        messages = conversation,
        functions = functions,
        temperature = 0.2
    )

    total_usage = add_dicts(total_usage, dict(response.usage))
    output = response.choices[0].message.content
    return {"output": output, "total_usage": total_usage}

correct_ques = []
total_usage = {'prompt_tokens': 0, 'completion_tokens': 0, 'total_tokens': 0}
for ques in list(json_ques.values())[0]:
    ques = str(ques).replace("'", '"')
    result = calculation_checking(ques)
    total_usage = add_dicts(total_usage, result["total_usage"])
    correct_ques.append(result["output"])
    print(result["output"])
    


AFTER FUNCTION {'total_tokens': 512, 'prompt_tokens': 485, 'completion_tokens': 27}
CALLING FUNCTION
QUERIES W = N[Integrate[1/x, {x, 1, 3}]]
good queries W = N[Integrate[1/x, {x, 1, 3}]]
RESULT FROM FUNCTION 1.0986122886681098
{'total_tokens': 1163, 'prompt_tokens': 1018, 'completion_tokens': 145}

AFTER FUNCTION {'total_tokens': 500, 'prompt_tokens': 476, 'completion_tokens': 24}
CALLING FUNCTION
QUERIES W = Integrate[1, {V, 1, 5}]
good queries W = Integrate[1, {V, 1, 5}]
RESULT FROM FUNCTION 4
{'total_tokens': 1123, 'prompt_tokens': 990, 'completion_tokens': 133}
['\n    {"question": "Calculate the work done during an isothermal process if the initial volume (V1) is 1 m^3, the final volume (V2) is 3 m^3 and the pressure is 1 atm.", "choices": {"a": "0.367 atm.m^3", "b": "0.693 atm.m^3", "c": "1.098 atm.m^3", "d": "1.386 atm.m^3"}, "answer": {"c": "1.098 atm.m^3"}}', '\n    {"question": "Calculate the work done during an isobaric process if the initial volume (V1) is 1 m^3, the fina

In [37]:
json.loads(correct_ques[0])

{'question': 'Calculate the work done during an isothermal process if the initial volume (V1) is 1 m^3, the final volume (V2) is 3 m^3 and the pressure is 1 atm.',
 'choices': {'a': '0.367 atm.m^3',
  'b': '0.693 atm.m^3',
  'c': '1.098 atm.m^3',
  'd': '1.386 atm.m^3'},
 'answer': {'c': '1.098 atm.m^3'}}